# Phi-4 Mini ONNX Paralelno Pozivanje Funkcija - Vodič

Ovaj vodič pokazuje kako koristiti Phi-4 Mini s ONNX Runtime GenAI za paralelno pozivanje funkcija. Pozivanje funkcija omogućuje modelu inteligentno korištenje vanjskih alata i API-ja na temelju korisničkih zahtjeva.

## Pregled

U ovom vodiču naučit ćete kako:
- Postaviti Phi-4 Mini s ONNX Runtime GenAI
- Definirati sheme funkcija za rezervaciju letova i hotela
- Koristiti vođenu generaciju s Lark gramatikom za strukturirani izlaz
- Izvršavati paralelna pozivanja funkcija za složene scenarije rezervacije putovanja

## Preduvjeti

Prije pokretanja ovog vodiča, osigurajte da imate:
- Preuzet Phi-4 Mini ONNX model
- Instaliran paket `onnxruntime-genai`
- Osnovno razumijevanje koncepta pozivanja funkcija


## Korak 1: Uvoz potrebnih biblioteka

Prvo ćemo uvesti potrebne biblioteke za implementaciju poziva funkcija.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Korak 2: Postavljanje i konfiguracija modela

Sada ćemo konfigurirati Phi-4 Mini ONNX model. Obavezno zamijenite putanju stvarnim direktorijem vašeg modela.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Korak 3: Postavljanje parametara generiranja

Postavite parametre generiranja kako biste kontrolirali ponašanje modela pri generiranju izlaza. Ove postavke osiguravaju determinističke i usmjerene odgovore za pozivanje funkcija.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Korak 4: Definirajte dostupne funkcije

Ovdje definiramo funkcije koje naš AI asistent može pozvati. U ovom primjeru imamo dvije funkcije vezane uz putovanja:
1. **booking_flight_tickets**: Za rezervaciju letova između zračnih luka
2. **booking_hotels**: Za rezervaciju hotelskog smještaja

Definicije funkcija slijede format OpenAI-ove sheme za pozivanje funkcija.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Korak 5: Pomoćne funkcije za generiranje gramatike

Ove pomoćne funkcije pretvaraju naše definicije funkcija u Lark gramatički format, koji koristi ONNX Runtime GenAI za vođeno generiranje. Ovo osigurava da model generira ispravne pozive funkcija u odgovarajućem JSON formatu.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Korak 6: Testiranje generiranja gramatike

Testirajmo naše funkcije za generiranje gramatike kako bismo vidjeli kako pretvaraju definicije alata u odgovarajući format.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Korak 7: Pripremite sistemski prompt i generator

Sada ćemo kreirati sistemski prompt koji obavještava model o dostupnim alatima i postaviti generator s parametrima za vođenu generaciju.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Korak 8: Inicijalizirajte Generator s Vodećom Generacijom

Sada ćemo stvoriti generator s našim konfiguriranim parametrima i primijeniti Lark gramatiku za vođenu generaciju.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Korak 9: Testiranje paralelnog poziva funkcija

Sada ćemo testirati našu postavku s kompleksnim scenarijem rezervacije putovanja koji zahtijeva pozivanje više funkcija.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generiranje poziva funkcija

Model će sada generirati strukturirane pozive funkcija na temelju korisničkog zahtjeva. Zahvaljujući vođenoj generaciji, izlaz će biti u valjanom JSON formatu koji se može izravno izvršiti.

**Očekivani izlaz**: Model bi trebao generirati pozive funkcija za:
1. `booking_flight_tickets` s detaljima za Peking (PEK) do Pariza (CDG)
2. `booking_hotels` s detaljima za smještaj u Parizu

Pokrenite ćeliju ispod kako biste vidjeli generaciju uživo:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Zaključak

🎉 **Čestitamo!** Uspješno ste implementirali paralelno pozivanje funkcija s Phi-4 Mini koristeći ONNX Runtime GenAI.

### Što ste naučili:

1. **Postavljanje modela**: Kako konfigurirati Phi-4 Mini s ONNX Runtime GenAI
2. **Definicija funkcija**: Kako definirati sheme funkcija za AI pozivanje funkcija
3. **Vođeno generiranje**: Kako koristiti Lark gramatiku za generiranje strukturiranog izlaza
4. **Paralelno pozivanje funkcija**: Kako upravljati složenim zahtjevima koji zahtijevaju više poziva funkcija

### Ključne prednosti:

- ✅ **Strukturirani izlaz**: Vođeno generiranje osigurava valjane JSON pozive funkcija
- ✅ **Paralelna obrada**: Upravljanje višestrukim pozivima funkcija u jednom zahtjevu
- ✅ **Visoke performanse**: ONNX Runtime omogućuje optimizirano izvođenje
- ✅ **Fleksibilna shema**: Jednostavno dodavanje ili izmjena definicija funkcija

### Resursi:

- [Phi-4 Mini Dokumentacija](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentacija](https://onnxruntime.ai/docs/genai/)
- [Najbolje prakse za pozivanje funkcija](https://platform.openai.com/docs/guides/function-calling)



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
